## 文本生成

#### 一、需要注意的点：

* 1、`LTP` 使用的是 `cuda` ，在进行分词的时候遇到了内存不足的情况，所以采用 `jieba` 分词。

* 2、使用 `conda` 虚拟环境时，如果爆出 `no jieba` 使用 `canda install jieba` 解决 

#### 二、可能影响模型效果的步骤：

* 1、文本打标签的步骤，比如 `<bos> <eos> <pad>` 等标签。


* 2、此时需要进 `pad_sq` 填充，造成文末很多 `<pad>` 标签进行补齐。


* 3、预测效果不好的原因会受训练时训练数据 `inputs` 长度的影响。


* 4、过少的 `epoch` 最后生成的文本预测结果会出现重复文字的情况


* 5、神经网络隐藏层的神经元数量和 `LSTM` 隐藏层的熟练对最后模型的学习有影响，神经元少，隐藏层少，最后的预测结果会出现重复文字的情况，主要 `epoch` 要多

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import numpy as np
# from ltp import LTP
import jieba

In [3]:
# Constants
BOS_TOKEN = "<bos>" ## 句首标记
EOS_TOKEN = "<eos>" ## 句尾标记
PAD_TOKEN = "<pad>" ## 补齐标记

In [4]:
from collections import defaultdict, Counter

class Vocab:
    def __init__(self, tokens=None):
        self.idx_to_token = list()
        self.token_to_idx = dict()

        if tokens is not None:
            if "<unk>" not in tokens:
                tokens = tokens + ["<unk>"]
            for token in tokens:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
            self.unk = self.token_to_idx['<unk>']

    @classmethod
    def build(cls, text, min_freq=1, reserved_tokens=None):
        token_freqs = defaultdict(int)
        for sentence in text:
            for token in sentence:
                token_freqs[token] += 1
        uniq_tokens = ["<unk>"] + (reserved_tokens if reserved_tokens else [])
        uniq_tokens += [token for token, freq in token_freqs.items() \
                        if freq >= min_freq and token != "<unk>"]
        
        return cls(uniq_tokens)   ## 返回cls 对象，到时候就可以通过这个cls 来调用 Vocab 类中的其他方法
                                  ## 后面调用 build ，并且将返回值设为 vocab = cls(uniq_tokens) ，之后调用vocab，会默认带有uniq_tokens参数
                                 ## 此 uniq_tokens 参数会传给 __init__ 中的 tokens。
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, token):
        return self.token_to_idx.get(token, self.unk)

    def convert_tokens_to_ids(self, tokens):
        return [self[token] for token in tokens] 

    def convert_ids_to_tokens(self, indices):
        return [self.idx_to_token[index] for index in indices]

#### 添加自定义词典

In [5]:
## 添加自定义词典
jieba.load_userdict('./dict.txt')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.526 seconds.
Prefix dict has been built successfully.


#### 尝试将一段文本分词

In [6]:
## 尝试将一段文本分词
' '.join(jieba.cut('第一章陨落的天才')).split()

['第一章', '陨落', '的', '天才']

#### 加载文本，然后进行分词，写入到分词文件中

In [31]:
## 加载文本，然后进行分词，写入到分词文件中

def text_to_words(path ='./斗破苍穹.txt',path2 ='./nlp_test1.txt' ):
    
    with open(path,encoding='UTF-8',mode='r') as f:
    #     document = f.read()[0:370000].replace(' ','').replace('“','').replace('”','').replace('，','') ## 尝试去除一些标点符号
        document = f.read()[0:670000].replace(' ','')

        document_cut = jieba.cut(document)

        #print  ' '.join(jieba_cut)  //注意：如果打印结果，则分词效果消失，后面的result无法显示
        result = ' '.join(document_cut)
        with open(path2, encoding='UTF-8',mode='w') as f2:
            f2.write(result)

    path = './nlp_test1.txt'
    
    return path

In [32]:
# path= './斗破苍穹_辉哥版.txt'
path= './斗破苍穹.txt'

path = text_to_words(path)

In [33]:
## 查看一下分词文件
with open(path, 'r',encoding='UTF-8') as f:
#     text = f.read()  ## 赋值之后，下面的print 输出就为空了
    print(f.read())

第一章 陨落 的 天才 
 
 “ 斗之力 ， 三段 ！ ” 
 
 望着 测验 魔石碑 上面 闪亮 得 甚至 有些 刺眼 的 五个 大字 ， 少年 面 无表情 ， 唇角 有着 一抹 自嘲 ， 紧握 的 手掌 ， 因为 大力 ， 而 导致 略微 尖锐 的 指甲 深深 的 刺进 了 掌心 之中 ， 带来 一阵阵 钻心 的 疼痛 … 
 
 “ 萧炎 ， 斗之力 ， 三段 ！ 级别 ： 低级 ！ ” 测验 魔石碑 之 旁 ， 一位 中年男子 ， 看 了 一眼 碑 上 所 显示 出来 的 信息 ， 语气 漠然 的 将 之 公布 了 出来 … 
 
 中年男子 话 刚刚 脱口 ， 便是 不出意外 的 在 人头 汹涌 的 广场 上带 起 了 一阵 嘲讽 的 骚动 。 
 
 “ 三段 ？ 嘿嘿 ， 果然 不出 我 所料 ， 这个 “ 天才 ” 这 一年 又是 在 原地踏步 ！ ” 
 
 “ 哎 ， 这 废物 真是 把 家族 的 脸 都 给 丢光 了 。 ” 
 
 “ 要不是 族长 是 他 的 父亲 ， 这种 废物 ， 早就 被 驱赶 出 家族 ， 任其 自生自灭 了 ， 哪 还有 机会 待 在 家族 中 白吃白喝 。 ” 
 
 “ 唉 ， 昔年 那 名闻 乌坦城 的 天才少年 ， 如今 怎么 落魄 成 这般 模样 了 啊 ？ ” 
 
 “ 谁 知道 呢 ， 或许 做 了 什么 亏心事 ， 惹 得 神灵 降怒 了 吧 … ” 
 
 周围 传来 的 不屑 嘲笑 以及 惋惜 轻叹 ， 落 在 那 如 木桩 待 在 原地 的 少年 耳 中 ， 恍如 一根根 利刺 狠狠 的 扎 在 心脏 一般 ， 让 得 少年 呼吸 微微 急促 。 
 
 少年 缓缓 抬起头来 ， 露出 一张 有些 清秀 的 稚嫩 脸庞 ， 漆黑 的 眸子 木然 的 在 周围 那些 嘲讽 的 同龄人 身上 扫 过 ， 少年 嘴角 的 自嘲 ， 似乎 变得 更加 苦涩 了 。 
 
 “ 这些 人 ， 都 如此 刻薄 势力 吗 ？ 或许 是因为 三年 前 他们 曾经 在 自己 面前 露出 过 最 谦卑 的 笑容 ， 所以 ， 如今 想要 讨还 回去 吧 … ” 苦涩 的 一笑 ， 萧炎 落寞 的 转身 ， 安静 的 回到 了 队伍 的 最后 一排 ， 孤单 的 身影 ， 与 周围 的 世界 ， 有些 格格不入

#### lstm 和rnn 进行文本训练的时候不需要去除标点符号和引入停止符，但是在读取数据的时候会有多余的 \n 符号，这里进行过滤

In [19]:
## 测试，将文中的\n 去掉，会应用到 load_reuters 中
# text = []
# with open(path,encoding='UTF-8',mode='r') as f:
#     for i in f.readlines():
#         i = i.replace('\n', '').split()
#         text.append(i)

# print('分词过滤前text len为：',len(text))
# text  = [line for line in text if line != [] and  10 < len(line) ]
# print('分词过滤后text len为：',len(text))

In [20]:
## 测试，将文中的\n 去掉，会应用到 load_data 中
# text = []
# tmp_text = []
# with open(path,encoding='UTF-8',mode='r') as f:
#     for i,content in enumerate(f.readlines()):
#         ##  20 行为一个输入样本
#         if i% 10 == 0:
#             text.append(tmp_text)
#             tmp_text = []
            
#         else:
#             content = content.replace('\n','')
#             tmp_text.extend(content.split())
            

# print('分词过滤前text len为：',len(text))
# text  = [line for line in text if line != []]
# print('分词过滤后text len为：',len(text))

#### 加载数据

In [34]:
## 拿文中的每一段作为一个样本进行训练
def load_words_text(path):
    
    text = []

    with open(path,encoding='UTF-8',mode='r') as f:
        for i in f.readlines():
            i = i.replace('\n', '').split()
            text.append(i)
    text  = [line for line in text if line != [] ]
    
    # lowercase (optional)
#     segment = [[word.lower() for word in sentence] for sentence in segment]
    vocab = Vocab.build(text, reserved_tokens=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN])
    corpus = [vocab.convert_tokens_to_ids(sentence) for sentence in text]

    return corpus, vocab

In [35]:
# ## 读取文本的时候选用20行做一个样本（不是很必要）
# def load_reuters(path):
    
#     text = []
#     tmp_text = []
    
#     with open(path,encoding='UTF-8',mode='r') as f:
#         for i,content in enumerate(f.readlines()):
#             ##  20 行为一个输入样本
#             if i% 10 == 0:
#                 text.append(tmp_text)
#                 tmp_text = []

#             else:
#                 content = content.replace('\n','')
#                 tmp_text.extend(content.split())
                
#     text  = [line for line in text if line != []]
    
#     # lowercase (optional)
# #     segment = [[word.lower() for word in sentence] for sentence in segment]
# #     vocab = Vocab.build(text, reserved_tokens=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN])
#     vocab = Vocab.build(text)

#     corpus = [vocab.convert_tokens_to_ids(sentence) for sentence in text]

#     return corpus, vocab

#### 将数据转为 dataloader 数据类型

In [36]:
def get_loader(dataset, batch_size, shuffle=True):
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=dataset.collate_fn,
        shuffle=shuffle
    )
    return data_loader

#### 保存词向量

In [37]:
# def save_pretrained(vocab, embeds, save_path):
#     """
#     Save pretrained token vectors in a unified format, where the first line
#     specifies the `number_of_tokens` and `embedding_dim` followed with all
#     token vectors, one token per line.
#     """
#     with open(save_path, "w") as writer:
#         writer.write(f"{embeds.shape[0]} {embeds.shape[1]}\n")
#         for idx, token in enumerate(vocab.idx_to_token):
#             vec = " ".join(["{:.4f}".format(x) for x in embeds[idx]])
#             writer.write(f"{token} {vec}\n")
#     print(f"Pretrained embeddings saved to: {save_path}")

#### 如果batch_size 为1 ，下面函数的意义不大，如果batch_size 不为1，那么因为序列长度不一致的原因，需要进行补齐，需要用到如下函数

In [38]:
from torch.nn.utils.rnn import pad_sequence

class RnnlmDataset(Dataset):
    def __init__(self, corpus, vocab):
        self.data = []
        self.bos = vocab[BOS_TOKEN]
        self.eos = vocab[EOS_TOKEN]
        self.pad = vocab[PAD_TOKEN]
        for sentence in tqdm(corpus, desc="Dataset Construction"):
            # 模型输入：BOS_TOKEN, w_1, w_2, ..., w_n
#             input = [self.bos] + sentence                ## 给数据打上起始标签
            input = sentence[:-1]                          ## 不使用起始标签

            # 模型输出：w_1, w_2, ..., w_n, EOS_TOKEN
#             target = sentence + [self.eos]              ## 给数据打上结束标签
            target = sentence[1:]                         ## 不使用结束标签

            self.data.append((input, target))             

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]

    def collate_fn(self, examples):
        # 从独立样本集合中构建batch输入输出
        inputs = [torch.tensor(ex[0]) for ex in examples]
        targets = [torch.tensor(ex[1]) for ex in examples]
        # 对batch内的样本进行padding，使其具有相同长度
        ## 即使你设置batch 为1 ，最好也进行补齐，因为操作之后维度会变成2D,不进行补齐维度是1D，在训练模型的时候需要进行一个torch.unse..操作
        inputs = pad_sequence(inputs, batch_first=True, padding_value=self.pad)
        targets = pad_sequence(targets, batch_first=True, padding_value=self.pad)
        return (inputs, targets)

In [39]:
## 循环神经网络语言模型
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,num_layers=2):
        super(RNNLM, self).__init__()
        # 词嵌入层
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 循环神经网络：这里使用LSTM
        self.rnn = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        # 输出层
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # 计算每一时刻的隐含层表示
        hidden, _ = self.rnn(embeds)
        output = self.output(hidden)

        return output

In [40]:
## 循环神经网络语言模型
# class RNNLM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim,num_layers=2):
#         super(RNNLM, self).__init__()
#         # 词嵌入层
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         # 循环神经网络：这里使用LSTM
#         self.rnn = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
#         # 输出层
#         self.output = nn.Linear(hidden_dim, vocab_size)

#     def forward(self, inputs):
#         embeds = self.embeddings(inputs)
#         # 计算每一时刻的隐含层表示
#         hidden, _ = self.rnn(embeds)
#         output = self.output(hidden)


#         return output

In [41]:
embedding_dim = 128
hidden_dim = 256
batch_size = 1
num_epoch = 20

# 读取文本数据，构建FFNNLM训练数据集（n-grams）
corpus, vocab = load_words_text(path)
dataset = RnnlmDataset(corpus, vocab)
data_loader = get_loader(dataset, batch_size)

Dataset Construction:   0%|          | 0/10148 [00:00<?, ?it/s]

In [42]:
len(data_loader)

10148

In [43]:
## 交叉熵损失函数
ce_loss  = nn.CrossEntropyLoss()
# 构建RNNLM，并加载至device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNLM(len(vocab), embedding_dim, hidden_dim)
model.to(device)
model

RNNLM(
  (embeddings): Embedding(20313, 128)
  (rnn): LSTM(128, 256, num_layers=2, batch_first=True)
  (output): Linear(in_features=256, out_features=20313, bias=True)
)

In [44]:
# 使用Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(num_epoch):
    total_loss = 0
    for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
        inputs, targets = [x.to(device) for x in batch]
        optimizer.zero_grad()
        output = model(inputs)
        loss = ce_loss(output.view(-1, output.shape[-1]), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()        ## 记录总损失
    print(f"Loss: {total_loss:.2f}")

# save_pretrained(vocab, model.embeddings.weight.data, "rnnlm.vec") ## 保存词向量

Training Epoch 0:   0%|          | 0/10148 [00:00<?, ?it/s]

Loss: 61498.21


Training Epoch 1:   0%|          | 0/10148 [00:00<?, ?it/s]

Loss: 52176.89


Training Epoch 2:   0%|          | 0/10148 [00:00<?, ?it/s]

...

Loss: 12170.30


Training Epoch 33:   0%|          | 0/10148 [00:00<?, ?it/s]

Loss: 12017.48


Training Epoch 34:   0%|          | 0/10148 [00:00<?, ?it/s]

KeyboardInterrupt: 

### 预测函数

我们不能单单用一个词来预测下一个词，这样就成了 n-gram 了，所以我们在预测的时候也不能只用一个词来预测下一个。而要用前面的所有作为条件预测下一个。

因为构造的 inputs 和 target 是按照 sentence[:-1] 和 sentence[1:] 来进行构造的。滑动的size 为1，所以对于模型最终的预测结果来说，真正起到预测的只是最后一个词。

text 为输入的文本，如果想要让文本自动的生成，你需要不断的进行预测，第一次预测，将输入的 text 转成 ids 形式，赋值给words_index这是输入样本，输入之后，将预测结果中真正有意义的最后一个词添加到words_index 中，反复进行这样的操作，最后就能进行文本生成。

In [45]:
def predict(model, text, next_words=20):
      
    words_index = vocab.convert_tokens_to_ids(text)
    ## 将上面的index 转成tensor形式
    words_index = torch.from_numpy(np.array([int(line) for line in words_index if line != 1])).long().to('cuda') 

    model.eval()     
    device = 'cuda'
    model.to(device)
    
    with torch.no_grad():

        for i in range(0, next_words):
            
            y_pred = model(torch.unsqueeze(words_index,dim=0))

            pre_word_index = y_pred.argmax(dim=2)
            words_index = torch.cat([words_index,pre_word_index[0][[-1]]],dim=0)
            
    return words_index

In [102]:
text  = ['你','真的','是','萧炎','吗','?']
text

['你', '真的', '是', '萧炎', '吗', '?']

In [103]:
words_index = vocab.convert_tokens_to_ids(text)
words_index

[581, 1886, 108, 51, 196, 623]

In [106]:
pre_words  = predict(model,text,next_words=550)

In [33]:
# pre_words

In [108]:
''.join(vocab.convert_ids_to_tokens(pre_words))

'你真的是萧炎吗?”望着那满脸懒散的纳兰嫣然，只得满脸好奇的微笑道。“没事，如今我还想问二品炼药师，看你没有多少反映，方才有什么东西！”纳兰肃无辜的道。“这种潜在!”手掌一挥，萧厉冰寒的轻吸了一口凉气，这种能够让人勉强的大致效果，要强上，在自己面前的情况下，让得他极为的存在，显然，他也并没有丝毫留手的清秀少年。这种潜力，能够拿出让自己取回金币啊！”对于萧炎这模样，不由得有些茫然的道：“你知道紫火大人没有？”见到萧炎点头，对面的古特话来，却是有些羡慕，虽然话依然年轻的建议，不过在那平日里沉默寡言的刁蛮之下，可却都是隐藏着软了下来。不过，毕竟这斗技也是个毫不怜悯的地方，绝对不可能让得小姐心中有些不可能抗衡。不过比起萧炎的灵魂灵蛇，却还远远比不上并且吧？说不定日后只能绝对比不上任何魔兽的团长，那将至少是何种凄惨地强横。他也并没有看不起这东西。这怎么可能对自己的温和啊，实在是有些太好。这如何不让地老者，其实都是在炼药师公会寻找到对方地实力。定要如何等级标志地潜力。别说是如何。自己的东西。却是让得很多人都是一位小心地察觉到对方自己给药老地东西。这里应该并不太大地危险性。嗯。她明显是一名九星斗者蛇人。谁会不敢帮忙。按照常理期间。拥有何种地号召力。毕竟说不得还会有可能。便又是高兴。便是让自己炼药大师。现在面对着那行将就木地人影。这里可是在斗者四星之下。拥有何种地号召力。”药老莫名地眼睛有些不解地意味。淡淡地道：“这里地损失。恐怕我们就得放在这里有关吧？”任由中年人笑吟吟的道。萧炎皱眉道：“这要过来炼制出来地。”微微一笑。药老撇嘴道：“全都是漠铁佣兵团地人后。你需要地阶的斗技。越是拖延。你若是没有多少地感觉。只要你能在这里。你。一旦日后地底线与你。”高台之上。眼瞳之中。眼瞳中掠过一抹诧异。一件大声漆黑的泛在半空中。粗略那名黑色泛着森冷的轻响。将晴叶地淡淡一笑。从中在天际暴响而之间。双脚稳稳的老者在树枝上地白色粉末。忍不住地从一枚黑色戒指高速旋转而去。右手之上。随着一道轻微的声响。隐隐有着几声。犹如离弦地黑暗。渗透开来。'

#### 保存模型

In [56]:
save_path = './lstm_net_epoch33'
# 保存整个网络
# torch.save(model2, save_path) 

# 保存网络中的参数, 速度快，占空间少
torch.save(model.state_dict(),save_path)

#### 加载模型

In [ ]:
# model3 = RNNLM(len(vocab), embedding_dim, hidden_dim)
# model3

In [ ]:
#针对上面一般的保存方法，加载的方法分别是：
# model_dict=torch.load(save_path)

# model3.load_state_dict(torch.load(save_path))

## --------------------------------------------------- 调试部分 --------------------------------------------------- 

In [ ]:
vocab.convert_ids_to_tokens(inputs[0])

In [ ]:
words_index = vocab.convert_tokens_to_ids(text)
words_index

In [ ]:
words_index = torch.from_numpy(np.array([int(line) for line in words_index if line != 1])).long().to('cuda')
words_index

In [ ]:
torch.unsqueeze(words_index,dim=0)

In [ ]:
with torch.no_grad():
    y_pred = model(torch.unsqueeze(words_index,dim=0))

In [ ]:
y_pred = y_pred.argmax(dim=2)
y_pred

In [ ]:
vocab.convert_ids_to_tokens([y_pred[0][-1]])

In [ ]:
y_pred[0][[-1]]

In [ ]:
words_index

In [ ]:
words_index = torch.cat([words_index,y_pred[0][[-1]]],dim=0)
words_index

In [ ]:
' '.join(vocab.convert_ids_to_tokens(words_index))

In [ ]:
# ' '.join(vocab.convert_ids_to_tokens(inputs[0]))